In [1]:
# noexport

!typech efficacy_utils.ipynb

retention_utils.py:448: error: Name 'get_ipython' is not defined
retention_utils.py:511: error: Name 'get_ipython' is not defined
retention_utils.py:515: error: Name 'get_ipython' is not defined
efficacy_utils.py:40: error: Name 'get_condition_to_installs_for_random_assignment_abtest' already defined on line 27
efficacy_utils.py:163: error: Name 'get_ipython' is not defined
efficacy_utils.py:180: error: Name 'get_ipython' is not defined
efficacy_utils.py:184: error: Name 'get_ipython' is not defined
efficacy_utils.py:188: error: Name 'get_ipython' is not defined
efficacy_utils.py:192: error: Name 'get_ipython' is not defined
Found 9 errors in 2 files (checked 1 source file)


In [2]:
from retention_utils import *

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


<IPython.core.display.Javascript object>

R[write to console]: Error in withVisible({ : 找不到对象'plot_attrition_df'

R[write to console]: Error in coxph(formula = Surv(lifetime, attritioned) ~ Frequency, data = plot_attrition_df) : 
  没有"coxph"这个函数




Error in withVisible({ : 找不到对象'plot_attrition_df'

Error in coxph(formula = Surv(lifetime, attritioned) ~ Frequency, data = plot_attrition_df) : 
  没有"coxph"这个函数


In [3]:
import jsonmemo as jsonmemo_module
jsonmemo_module.set_lowmem(True)

In [4]:
#installs_with_experiment_vars = get_installs_with_experiment_vars()

In [5]:
#print(installs_with_experiment_vars[0])

In [6]:


def get_condition_to_installs_for_random_assignment_abtest():
  abtest_name = 'difficulty_selection_screen'
  groups = ['survey_nochoice_nothing', 'survey_nochoice_easy', 'survey_nochoice_medium', 'survey_nochoice_hard']
  condition_to_installs = get_conditions_to_install_list_in_abtest_unstrict(abtest_name)
  for k in list(condition_to_installs.keys()):
    if k not in groups:
      del condition_to_installs[k]
  return condition_to_installs


In [7]:


def get_condition_to_installs_for_random_assignment_abtest():
  abtest_name = 'difficulty_selection_screen_and_choose_difficulty_frequency'
  groups = ['survey', 'nodefault_forcedchoice_userchoice', 'survey_nochoice_nothing', 'survey_nochoice_easy', 'survey_nochoice_medium', 'survey_nochoice_hard']
  #condition_to_installs = get_conditions_to_install_list_in_abtest_unstrict(abtest_name)
  condition_to_installs = get_conditions_to_install_list_in_abtest(abtest_name, groups)
  for k in list(condition_to_installs.keys()):
    if k not in groups:
      del condition_to_installs[k]
  return condition_to_installs


In [8]:
@msgpackmemo1arg
def get_domain_to_intervention_to_session_lengths_for_install_olddata(install):
  seconds_on_domain_per_session = get_collection_for_install_olddata(install, 'synced:seconds_on_domain_per_session')
  interventions_active_for_domain_and_session = get_collection_for_install_olddata(install, 'synced:interventions_active_for_domain_and_session')
  domain_to_session_to_intervention = {}
  for x in interventions_active_for_domain_and_session:
    val = x.get('val')
    if val is None or type(val) == int:
      continue
    interventions_active = json.loads(val)
    if len(interventions_active) == 0:
      continue
    intervention_name = interventions_active[0]
    domain = x['key']
    session_id = x['key2']
    if domain not in domain_to_session_to_intervention:
      domain_to_session_to_intervention[domain] = {}
    domain_to_session_to_intervention[domain][session_id] = intervention_name
  
  domain_to_intervention_to_session_lengths = {}
  for x in seconds_on_domain_per_session:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_session')
      print(x)
      continue
    domain = x['key']
    session_id = x['key2']
    time_spent = x['val']
    if domain not in domain_to_session_to_intervention:
      continue
    intervention_name = domain_to_session_to_intervention[domain].get(session_id)
    if intervention_name is None:
      continue
    if domain not in domain_to_intervention_to_session_lengths:
      domain_to_intervention_to_session_lengths[domain] = {}
    if intervention_name not in domain_to_intervention_to_session_lengths[domain]:
      domain_to_intervention_to_session_lengths[domain][intervention_name] = []
    domain_to_intervention_to_session_lengths[domain][intervention_name].append(time_spent)
  return domain_to_intervention_to_session_lengths
  

In [9]:
# noexport

for x in get_collection_names():
  if 'per_day' in x and 'visits' not in x:
    print(x)
    break

edceda6041ef668213a96ba5_synced:seconds_on_domain_per_day


In [10]:
@msgpackmemo1arg
def get_epoch_to_domain_to_time_spent(install):
  seconds_on_domain_per_day = get_collection_for_install(install, 'synced:seconds_on_domain_per_day')
  output = {}
  for x in seconds_on_domain_per_day:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_day')
      print(x)
      continue
    domain = x['key']
    epoch = x['key2']
    seconds = x['val']
    if epoch not in output:
      output[epoch] = {}
    output[epoch][domain] = seconds
  return output

In [11]:
# noexport

condition_to_installs = get_condition_to_installs_for_random_assignment_abtest()


In [12]:
# noexport

for condition,installs in condition_to_installs.items():
  for install in installs:
    get_epoch_to_domain_to_time_spent(install)
    #print(sorted(get_epoch_to_domain_to_time_spent(install).keys()))
    break

In [13]:
def convert_date_to_epoch(date):
  #start_of_epoch = moment.now().timezone("US/Pacific").replace(years=2016, months=1, days=1, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0)
  start_of_epoch = moment.now().replace(years=2016, months=1, days=1, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0)
  year = int(date[0:4])
  month = int(date[4:6])
  day = int(date[6:8])
  #date_moment = moment.now().timezone("US/Pacific").replace(years=year, months=month, days=day, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0)
  date_moment = moment.now().replace(years=year, months=month, days=day, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0)
  return date_moment.diff(start_of_epoch).days

#import moment
#convert_date_to_epoch('20200527')

In [14]:
import math

def make_domain_to_daily_time_dataframe():
  output = []
  for condition,installs in condition_to_installs.items():
    #condition_to_lengths[condition] = []
    for install in installs:
      for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
        if epoch > 1608: # 2020/05/27
          continue
#         total_time_spent = sum(domain_to_time_spent.values())
#         output.append({
#           'user': install,
#           'epoch': epoch,
#           'time': total_time_spent,
#           'logtime': math.log(total_time_spent),
#           'condition': condition,
#         })
        for domain,total_time_spent in  domain_to_time_spent.items():
          output.append({
            'domain': domain,
            'user': install,
            'epoch': epoch,
            'time': total_time_spent,
            'logtime': math.log(total_time_spent),
            'condition': condition,
          })
  return to_dataframe(output)

In [15]:
# noexport

df = make_domain_to_daily_time_dataframe()

In [16]:
#print(df)

In [17]:
# noexport

%Rpush df

In [18]:
#%%R
#install.library

In [19]:
%%R

library(lme4)
#library(sjPlot)
library(lmerTest)
library(stargazer)


R[write to console]: 载入需要的程辑包：Matrix

R[write to console]: Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

R[write to console]: 
载入程辑包：‘lmerTest’


R[write to console]: The following object is masked from ‘package:lme4’:

    lmer


R[write to console]: The following object is masked from ‘package:stats’:

    step


R[write to console]: 
Please cite as: 


R[write to console]:  Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.

R[write to console]:  R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 




In [20]:
# %%R

# df$user <- as.factor(df$user)
# #df$domain <- as.factor(df$domain)
# df$condition <- as.factor(df$condition)
# df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard"))
# df$epoch <- as.factor(df$epoch)
# df$logtime <- as.numeric(df$logtime)
# df$time <- as.numeric(df$time)
# summary(df)



In [30]:
%%R

df$user <- as.factor(df$user)
df$domain <- as.factor(df$domain)
df$condition <- as.factor(df$condition)
#df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard", "survey", "nodefault_forcedchoice_userchoice"))
df$condition <- factor(df$condition, levels = c("survey", "survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard", "nodefault_forcedchoice_userchoice"))
df$epoch <- as.factor(df$epoch)
df$logtime <- as.numeric(df$logtime)
df$time <- as.numeric(df$time)
#summary(df)




In [22]:
%%R

#dfsub <- df[df$domain == 'www.youtube.com', ]
#show(summary(dfsub))
#show(summary(subset(df, domain='www.youtube.com')))
#dfsub <- subset(df, domain='www.youtube.com')
#show(summary(dfsub))

NULL


In [23]:
# %%R

# dfsub <- df
# show(summary(dfsub))
# #results <- lmer(logtime ~ condition + (1|user), data = df)
# #results <- lmer(logtime ~ condition + (1|user), data = dfsub)
# results <- lmer(logtime ~ condition + (1|user) + (1|domain), data = df)
# show(results)
# show(summary(results))
# class(results) <- "lmerMod"
# stargazer(results)

In [28]:
%%R

#show(df)

In [38]:
def convert_epoch_to_date(epoch):
  #start_of_epoch = moment.now().timezone("US/Pacific").replace(years=2016, months=1, days=1, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0)
  start_of_epoch = moment.now().replace(years=2016, months=1, days=1, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0)
  start_of_epoch.add(days=epoch)
  return start_of_epoch.format('YYYYMMDD')

import moment
convert_epoch_to_date(730)

'20171231'

In [36]:
min([x for x in df['epoch'].values if x > 0])

730

In [31]:
%%R

dfsub <- df[df$domain == 'www.facebook.com', ]
show(summary(dfsub))
#results <- lmer(logtime ~ condition + (1|user), data = df)
results <- lmer(logtime ~ condition + (1|user), data = dfsub)
#results <- lmer(logtime ~ condition + (1|user) + (1|domain), data = df)
show(results)
show(summary(results))
class(results) <- "lmerMod"
stargazer(results)

                   domain                            user           epoch      
 www.facebook.com     :77165   3d8f23613f3849ed266ace58:  341   1592   :  430  
 _.shoprintee.com     :    0   f9329b2116e3bd031cca0519:  281   1573   :  429  
 _antonia.gr          :    0   65180c485ea59d22fdc32a4d:  271   1587   :  423  
 _dmarc.adoreme.com   :    0   79f9aa9fe47ba2f0fa59ba73:  270   1599   :  422  
 _dnslink.your.website:    0   71da006ec218a9b8e3eb317e:  268   1608   :  422  
 _kunden.suedcom.info :    0   26c1461ecb3d3dd841ecbaac:  266   1571   :  421  
 (Other)              :    0   (Other)                 :75468   (Other):74618  
      time          logtime                                   condition    
 Min.   :    1   Min.   : 0.000   survey                           :12837  
 1st Qu.:  115   1st Qu.: 4.745   survey_nochoice_nothing          :11401  
 Median :  450   Median : 6.109   survey_nochoice_easy             :13963  
 Mean   : 1146   Mean   : 5.823   survey_nochoice_medium

In [ ]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      condition_to_lengths[condition].append(total_time_spent)
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [ ]:
# noexport
# bad


condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for domain,intervention_to_session_lengths in get_domain_to_intervention_to_session_lengths_for_install_olddata(install).items():
      if domain != 'www.youtube.com':
        continue
      for intervention,session_lengths in intervention_to_session_lengths.items():
        for session_length in session_lengths:
          condition_to_lengths[condition].append(session_length)


In [ ]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    all_lengths_for_user = []
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      all_lengths_for_user.append(total_time_spent)
    if len(all_lengths_for_user) == 0:
      continue
    condition_to_lengths[condition].append(np.median(all_lengths_for_user))
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [ ]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      if 'www.facebook.com' not in domain_to_time_spent:
        continue
      time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      condition_to_lengths[condition].append(time_on_facebook)

In [ ]:
# noexport
from scipy.stats import ttest_ind

condition_to_mlengths = {}
for condition,session_lengths in condition_to_lengths.items():
  print(condition, np.mean(session_lengths))
  condition_to_mlengths[condition] = np.mean(session_lengths) / 60

#print(ttest_ind(condition_to_lengths['survey_nochoice_hard'], condition_to_lengths['survey_nochoice_nothing']))
print(ttest_ind(condition_to_lengths['survey_nochoice_hard'], condition_to_lengths['survey_nochoice_easy']))
#print(ttest_ind(condition_to_lengths['survey_nochoice_easy'], condition_to_lengths['survey_nochoice_medium']))


In [ ]:
# noexport

from plot_utils import *


In [ ]:
# noexport

#from plot_utils import *
plot_dict_as_bar(condition_to_mlengths, title='Total minutes spent daily on sites', xlabel='Difficulty', ylabel='Minutes spent daily on sites', remap_labels={'survey_nochoice_nothing': 'Nothing', 'survey_nochoice_easy': 'Easy', 'survey_nochoice_medium': 'Medium', 'survey_nochoice_hard': 'Hard', 'survey': 'Experience Sampling', 'nodefault_forcedchoice_userchoice': 'Onboarding Choice'})

In [ ]:
# for install in installs_with_experiment_vars:
#   #seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
#   #interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
#   res = get_domain_to_intervention_to_session_lengths_for_install(install)
#   #if len(res) > 0:
#   #  print(res)
#   #  break